<a href="https://colab.research.google.com/github/Gladiator07/Natural-Language-Processing/blob/main/Basics/Text-Preprocessing/spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spacy Tutorial

### References
- [Overview of Spacy](https://www.analyticsvidhya.com/blog/2020/03/spacy-tutorial-learn-natural-language-processing/)
- [Comprehensive article](https://www.machinelearningplus.com/spacy-tutorial-nlp/)

## Spacy's Processing Pipeline

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/03/spacy_pipeline.png)

The first step for a text string, when working with spaCy, is to pass it to an NLP object. This object is essentially a pipeline of several text pre-processing operations through which the input text string has to go through.

In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [2]:
# create an nlp object
doc = nlp("He went to play basketball")

In [3]:
# seeing the active pipelines
nlp.pipe_names

['tagger', 'parser', 'ner']

In [4]:
# disable pipeline components (if not required, can save up the computation)
nlp.disable_pipes('tagger', 'parser')

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7fb2e6b0fd90>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7fb2e6cd9980>)]

In [5]:
nlp.pipe_names

['ner']

## 1.POS Tagging (Part-of-Speech)

In English grammar, the parts of speech tell us what is the function of a word and how it is used in a sentence. Some of the common parts of speech in English are Noun, Pronoun, Adjective, Verb, Adverb, etc.

POS tagging is the task of automatically assigning POS tags to all the words of a sentence. It is helpful in various downstream tasks in NLP, such as feature engineering, language understanding, and information extraction.

In [8]:
nlp = spacy.load('en_core_web_sm')

# create an nlp object
doc = nlp("He went to play basketball")

# iterate over the tokens
for token in doc:
    print(token.text, "-->", token.pos_)
    # print(token)

basketball --> NOUN
